Plot the QSGW/QSGW^ band structure and highlight the BSE transition space, 
depending on if the material has completed the entire workflow.

The example materials listed under 'db_entry' clearly demonstrate how the band space for the BSE is selected and how valence band manifolds are filled when they are close to the 10 eV energy cutoff.

In [ ]:
"""
START USER INPUT 
"""

db_name  = "../../questaal_database"
db_entry = "BiIO_icsd_391354_nsites_6.json" # examples: "C_icsd_190650_nsites_2.json", "BiIO_icsd_391354_nsites_6.json"

figsize  = [3, 3] # figure size (width, height) in inches

"""
END USER INPUT 
"""

# external imports
import os
import sys
import numpy as np
import matplotlib.pyplot as plt

# internal imports
from qsgw_workflow.utils.helper import load_db_entry, plot_bs

# Noctua 2 has no LaTeX...
plt.rcParams["text.usetex"] = False
plt.rcParams["font.family"] = "DejaVu Serif"

# load the files
db_path = os.path.join(db_name, db_entry)
if os.path.exists(db_path):
    print(f"Loading the following database entry: {db_path:s}")
    cse = load_db_entry(db_path)
else:
    sys.exit("The given database name or database entry does not exist!")
    
# check if the calculation finished
qsgw_flag = False # what band structure do we plot, QSGW or QSGW^?
if cse.parameters["finish"] == False:
    if cse.parameters["qsgw_flag"] == True:
        qsgw_flag = True
        print("Using the QSGW results, i.e., the QSGW^ has not finished or has crashed.")
    else:
        sys.exit("The calculation has not finished or has crashed!")
if cse.parameters["metal_flag_qsgw"] == True:
    sys.exit("\nThe material is a metal!")

# is everything converged?
if cse.parameters["qsgw_kpt_conv_error_flag"] == 1:
    print("The QSGW k-grid convergence stopped at the DFT k-grid.")
if cse.parameters["qsgw_scf_conv_error_flag"] == 1:
    print("The QSGW self-consistency loop did not converge.")
if not qsgw_flag:
    if cse.parameters["qsgwbse_scf_conv_error_flag"] == 1:
        print("The QSGW^ self-consistency loop did not converge.")

# print some information about the QSGW^
nv = cse.parameters["nv"]
nc = cse.parameters["nc"]
kpts = cse.parameters["qsgw_kpts"]
if not qsgw_flag:
    gap = cse.data["gap_qsgwbse"]
    print(f"QSGW^ parameters and results:\nk-grid=({kpts[0]:d}x{kpts[1]:d}x{kpts[2]:d}), NV={nv:d}, NC={nc:d} -> gap={gap:2.3f} eV.")
else:
    print(f"QSGW^ parameters:\nk-grid=({kpts[0]:d}x{kpts[1]:d}x{kpts[2]:d}), NV={nv:d}, NC={nc:d}.")

# QSGW/QSGW^ band structure and highlight all bands included in the BSE Hamiltonian
fig, ax = plt.subplots(figsize=figsize)
if not qsgw_flag:
    bs = cse.data["bs_qsgwbse"]
else:
    bs = cse.data["bs_qsgw"]
plot_bs(ax, bs, lcs="k-", deco=True)
vbm_idx = cse.parameters["vbm_idx"]
bs_paths = bs["bs_paths"]
x0 = 0
ymin = np.inf
ymax = -np.inf
for path in bs_paths:
    nk = path["nk"]
    x = np.arange(x0, x0 + nk)
    for i in range(vbm_idx + 1 - nv, vbm_idx + nc + 1):
        y = np.array(path["bands"])[:, i]
        curr_min = np.min(y)
        curr_max = np.max(y)
        ymin = np.min([ymin, curr_min])
        ymax = np.max([ymax, curr_max])
        ax.plot(x, y, "r--")
    x0 = x[-1]
ax.set_ylim([ymin - 2, ymax + 2])
ax.set_title(db_entry)
fig.align_xlabels()
fig.tight_layout()